In [2]:
wos_path = '/Users/dong/Desktop/Article_1_prepare/第一次文献keyword分析/wos_record.txt'
scopus_path = '/Users/dong/Desktop/Article_1_prepare/第一次文献keyword分析/scopus_record.csv'
pubmed_path = '/Users/dong/Desktop/Article_1_prepare/第一次文献keyword分析/pubmed_record.txt'
import pandas as pd

# 1
## Identifying, union, merge articles from PubMed, Scopus, and WoS

In [ ]:
def parse_scopus(filepath):
    """Parse Scopus CSV data and adjust the column names."""
    df = pd.read_csv(filepath)
    # 重命名列以符合期望的数据结构
    df.rename(columns={
        'Source title': 'Journal',  # 确保将'Source title'重命名为'Journal'
        'Title': 'Title',
        'Authors': 'Authors',
        'Year': 'Year',
        'DOI': 'DOI'
    }, inplace=True)
    df['Source'] = 'Scopus'  # 添加数据源列
    return df

def parse_pubmed(filepath):
    records = []
    with open(filepath, 'r', encoding='utf-8') as file:
        article = {}
        for line in file:
            if line.startswith('PMID-'):
                if article:
                    records.append(article)
                article = {'Source': 'PubMed'}
            elif line.startswith('TI  - '):
                article['Title'] = line.replace('TI  - ', '').strip()
            elif line.startswith('AU  - '):
                article['Authors'] = line.replace('AU  - ', '').strip()
            elif line.startswith('JT  - '):
                article['Journal'] = line.replace('JT  - ', '').strip()
            elif line.startswith('DP  - '):
                article['Year'] = line.replace('DP  - ', '').strip().split()[0]
            elif line.startswith('LID - ') and '[doi]' in line:
                article['DOI'] = line.replace('LID - ', '').strip().split()[0].replace('[doi]', '')
        if article:
            records.append(article)
    return pd.DataFrame(records)

def parse_wos(filepath):
    """Parse Web of Science data ensuring complete title capture and handling multiline entries for Title specifically."""
    records = []
    with open(filepath, 'r', encoding='utf-8') as file:
        article = {}
        current_key = None  # Current field being parsed
        for line in file:
            line = line.strip()
            if line.startswith('PT '):  # Start of a new record
                if article:
                    records.append(article)
                article = {'Source': 'WoS'}  # Initialize a new record
                current_key = None
            if line.startswith('TI '):  # Title may continue on multiple lines
                current_key = 'Title'
                article['Title'] = line[3:].strip()  # Initialize title
            elif line.startswith('AU '):  # Authors
                current_key = 'Authors'
                article['Authors'] = line[3:].strip()
            elif line.startswith('SO '):  # Journal name
                current_key = 'Journal'
                article['Journal'] = line[3:].strip()
            elif line.startswith('PY '):  # Publication year
                current_key = 'Year'
                article['Year'] = line[3:].strip()
            elif line.startswith('DI '):  # DOI
                current_key = 'DOI'
                article['DOI'] = line[3:].strip()
            elif current_key == 'Title' and not line.startswith('AU ') and not line.startswith('SO ') and not line.startswith('PY ') and not line.startswith('DI '):
                # Continue appending to the title if no new field tag is detected
                article['Title'] += ' ' + line

        if article:  # Append the last processed record
            records.append(article)

    return pd.DataFrame(records)




def standardize_df_columns(df, standard_cols):
    # Add missing columns with default values
    missing_cols = set(standard_cols) - set(df.columns)
    for col in missing_cols:
        df[col] = None  # Assign None or an appropriate default value
    # Reorder and select the standard columns
    return df[standard_cols]

# Parse data
pubmed_data = parse_pubmed(pubmed_path)
scopus_data = parse_scopus(scopus_path)
wos_data = parse_wos(wos_path)

# Ensure consistent column names
standard_cols = ['Title', 'Authors', 'Journal', 'Year', 'DOI', 'Source']
pubmed_data = standardize_df_columns(pubmed_data, standard_cols)
scopus_data = standardize_df_columns(scopus_data, standard_cols)
wos_data = standardize_df_columns(wos_data, standard_cols)

# Concatenate data
all_data = pd.concat([pubmed_data, scopus_data, wos_data], ignore_index=True)

# Checking and merging publications count
journal_counts = all_data['Journal'].value_counts().reset_index()
journal_counts.columns = ['Journal', 'Publications']
all_data = all_data.merge(journal_counts, on='Journal', how='left')

# Sort by publication counts and print the data
all_data.sort_values(by=['Publications', 'Journal'], ascending=[False,True], inplace=True)

# Save the final data
all_data.to_csv('sorted_publications.csv', index=False)



# 2
## Delete articles relate with "marine heatwave"

In [3]:
sorted_pd = pd.DataFrame(pd.read_csv('sorted_publications.csv'))
sorted_pd['Title'] = sorted_pd['Title'].str.lower()
sorted_pd = sorted_pd[sorted_pd['Title'].str.contains('marine')==False]
sorted_pd.to_csv('second_process.csv')

# 3
## Delete duplicates completely

In [9]:
temp_pd = pd.DataFrame(pd.read_csv('second_process.csv'))
temp_pd.dropna(subset=['Title'], inplace=True)
temp_pd.shape

(449, 8)

In [12]:
df = pd.read_csv('second_process.csv')

# 找到所有重复的标题
duplicates = df[df.duplicated('Title', keep=False)]

# 对重复的标题按来源进行聚合
title_sources = duplicates.groupby('Title')['Source'].agg(list).reset_index()


In [13]:
title_sources

,Title,Source
0,analysis of heatwaves based on the universal t...,"[WoS, Scopus]"
1,anthropogenic climate change exacerbates the r...,"[WoS, Scopus]"
2,behaviour broadens thermal safety margins on a...,"[WoS, Scopus]"
3,climate change over indonesia and its impact o...,"[WoS, Scopus]"
4,future changes in high and low flows under the...,"[WoS, Scopus]"
5,future exposure of rainfall and temperature ex...,"[WoS, Scopus]"
6,heatwaves in southeast asia and their changes ...,"[WoS, Scopus]"
7,how do disparate urbanization and climate chan...,"[WoS, Scopus]"
8,impacts of and adaptation to climate change on...,"[WoS, Scopus]"
9,interaction between heat wave and urban heat i...,"[WoS, Scopus]"


In [19]:
temp_pd.drop_duplicates(subset=['Title'],inplace = True)
temp_pd.to_csv('Third_process.csv')

# 4
## Fetch title related articles

In [28]:
temp_pd3 = pd.DataFrame(pd.read_csv('Third_process.csv'))
selected_articles = temp_pd3[temp_pd3['Yes No'] == 1]
selected_articles = selected_articles.iloc[:,1:]
selected_articles.to_csv('forth_process.csv', index = False)

In [39]:
detected_drop_articles = temp_pd3[temp_pd3['Yes No'] == 0]
detected_drop_articles.shape

(398, 9)